In [2]:
"""
This workflow will consist of 3 distinct task definitions:
 - ultimate partent task will maintain the flow of the overall state of the task
 - another task will manage the validation state/workflow for different data elements:
    - transaction
    - quote
 - another task will manage the workflow of each individual validation/default process for each of the data elements required... possible
 to use a single task definition for both txn and quote validation processes and select the action depending on field variable of the task?

Pre-requisites:
 - transaction portfolio to be in place
 - instrument for the workflow to be defined.
 - recipe to be defined.

dependencies
 - transaction check worker
 - transaction default worker
 - quote check worker
 - quote default worker
"""
# Set up
import os
import pandas as pd
import json
import uuid
import pytz
from datetime import timedelta
import datetime
import pprint

# API definitions
import lusid_workflow as wf
import lusid_workflow.models as wf_models
from lusid_workflow.models import *
from lusid_workflow.rest import ApiException
from lusidjam.refreshing_token import RefreshingToken

import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)


configuration = wf.Configuration(
    host="https://fbn-joe.lusid.com/workflow")
# note this doesn't really work locally yet, there's a change waiting to be merged to workflow SDK to enable that
# until then, upload to hosted jupyter server which will handle auth
configuration.access_token = RefreshingToken()

api_client = wf.ApiClient(configuration)

task_def_api = wf.TaskDefinitionsApi(api_client)
task_api = wf.TasksApi(api_client)
worker_api = wf.WorkersApi(api_client)

scope = "Demo_workflow_valuation"


In [3]:
# Helper functions.

def create_task_definition(scope, code, display_name, description, states, field_schema, initial_state, triggers, transitions, actions):

    task_definition = wf_models.CreateTaskDefinitionRequest(
        wf_models.ResourceId(
            scope,
            code
            ),
        display_name,
        description,
        states,
        field_schema,
        initial_state,
        triggers,
        transitions,
        actions
        )

    try:
        create_task_definition_request = task_def_api.create_task_definition(task_definition)
        logger.info("Created task definition for task: {code}")

    except ApiException as e:
        print(f"Exception raised: {e}")

c:\Users\JoeCockings\Repos\fbn-joe\.venv\Lib\site-packages\lusid_workflow\api\task_definitions_api.py:301: SyntaxWarning: invalid escape sequence '\-'
  raise ApiValueError("Invalid value for parameter `scope` when calling `delete_task_definition`, must conform to the pattern `/^[a-zA-Z0-9\-_]+$/`")  # noqa: E501
c:\Users\JoeCockings\Repos\fbn-joe\.venv\Lib\site-packages\lusid_workflow\api\task_definitions_api.py:309: SyntaxWarning: invalid escape sequence '\-'
  raise ApiValueError("Invalid value for parameter `code` when calling `delete_task_definition`, must conform to the pattern `/^[a-zA-Z0-9\-_]+$/`")  # noqa: E501
c:\Users\JoeCockings\Repos\fbn-joe\.venv\Lib\site-packages\lusid_workflow\api\task_definitions_api.py:476: SyntaxWarning: invalid escape sequence '\-'
  raise ApiValueError("Invalid value for parameter `scope` when calling `get_task_definition`, must conform to the pattern `/^[a-zA-Z0-9\-_]+$/`")  # noqa: E501
c:\Users\JoeCockings\Repos\fbn-joe\.venv\Lib\site-packages\

TypeError: can only concatenate str (not "NoneType") to str

In [ ]:
# Task definitions

# Ultimate parent task.